In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
def_train = spark.read.option("inferSchema","true").csv("Classification_Train.csv", header=True)
def_test = spark.read.option("inferSchema", "true").csv("Classification_Test.csv", header=True)

In [8]:
def_train = def_train.select("Education Level", "Married", "Salary Income", "Depressed")
def_test = def_test.select("Education Level", "Married", "Salary Income", "Depressed")

In [9]:
def_train = df_train.na.drop()
def_test = df_test.na.drop()

In [10]:
from pyspark.sql.functions import when

In [11]:
def_train = def_train.withColumn("Education Level", when(def_train["Education Level"] == "High", 2).when(def_train["Education Level"] == "Intermediate", 1).otherwise(0))
def_train = def_train.withColumn("Married", when(def_train["Married"] == "Yes", 1).otherwise(0))
def_train = def_train.withColumn("Depressed", when(def_train["Depressed"] == "Yes", 1).otherwise(0))

In [18]:
def_test = def_test.withColumn("Education Level", when(def_test["Education Level"] == "High", 2).when(def_test["Education Level"] == "Intermediate", 1).otherwise(0))
def_test = def_test.withColumn("Married", when(def_test["Married"] == "Yes", 1).otherwise(0))
def_test = def_test.withColumn("Depressed", when(def_test["Depressed"] == "Yes", 1).otherwise(0))

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
cols = def_train.columns
cols.remove("Depressed")
def_train = VectorAssembler(inputCols = cols, outputCol = "Features").transform(def_train)

IllegalArgumentException: 'Output column Features already exists.'

In [21]:
cols = def_test.columns
cols.remove("Depressed")
def_test = VectorAssembler(inputCols = cols, outputCol = "Features").transform(def_test)

In [22]:
from pyspark.ml.feature import StandardScaler

In [23]:
scaler = StandardScaler(inputCol = "Features", outputCol = "Scaler_Features")
def_train = scaler.fit(def_train).transform(def_train)

In [24]:
def_test = StandardScaler(inputCol = "Features", outputCol = "Scaler_Features").fit(def_test).transform(def_test)

In [25]:
from pyspark.ml.classification import LogisticRegression

In [26]:
model = LogisticRegression(featuresCol = "Scaler_Features", labelCol = "Depressed", maxIter = 10).fit(def_train)

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
prediction = model.transform(def_test)

In [29]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol = "rawPrediction", labelCol = "Depressed")

In [30]:
print("Accuracy {}".format(evaluator.evaluate(prediction)))

Accuracy 0.8706157923010207
